In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "/content/drive/MyDrive/Skin_Disease_Dataset.zip" /content/

In [ ]:
!unzip -q "/content/Skin_Disease_Dataset.zip" -d /content/skin_dataset

In [ ]:
!ls "/content/skin_dataset/skin Disease Dataset/kaggle"

test  train  val


In [ ]:
train_path = "/content/skin_dataset/skin Disease Dataset/kaggle/train"
val_path   = "/content/skin_dataset/skin Disease Dataset/kaggle/val"
test_path  = "/content/skin_dataset/skin Disease Dataset/kaggle/test"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.amp import autocast, GradScaler

In [ ]:
!mkdir -p /content/drive/MyDrive/models

In [ ]:
import os
import shutil
import numpy as np
from torchvision import datasets

NUM_CLASSES = 6
SAMPLES_PER_CLASS = 3000

train_dataset = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/train")
targets = np.array(train_dataset.targets)

balanced_dir = "/content/skin_dataset/skin Disease Dataset/kaggle/train_balanced"
os.makedirs(balanced_dir, exist_ok=True)

if os.path.exists(balanced_dir):
    shutil.rmtree(balanced_dir)

os.makedirs(balanced_dir)

for cls in range(NUM_CLASSES):
    cls_indices = np.where(targets == cls)[0]
    selected = np.random.choice(cls_indices, SAMPLES_PER_CLASS, replace=len(cls_indices)<SAMPLES_PER_CLASS)

    class_dir = os.path.join(balanced_dir, train_dataset.classes[cls])
    os.makedirs(class_dir, exist_ok=True)

    for j, idx in enumerate(selected):
     src_path = train_dataset.imgs[idx][0]

     filename = f"{j}_{os.path.basename(src_path)}"
     dst_path = os.path.join(class_dir, filename)

     shutil.copy(src_path, dst_path)


In [ ]:
!ls "/content/skin_dataset/skin Disease Dataset/kaggle"

test  train  train_balanced  val


In [ ]:
from torchvision import transforms
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights

weights = EfficientNet_B1_Weights.IMAGENET1K_V1

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transforms = transforms.Compose([
    transforms.Resize((240, 240)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    normalize
])

val_transforms = transforms.Compose([
    transforms.Resize((240, 240)),
    transforms.ToTensor(),
    normalize
])

In [ ]:
train_dataset = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/train_balanced", transform=train_transforms)
val_dataset   = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/val", transform=val_transforms)

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

Using device: cuda


In [ ]:
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights

weights = EfficientNet_B1_Weights.IMAGENET1K_V1
model_en = efficientnet_b1(weights=weights)

for param in model_en.features.parameters():
    param.requires_grad = False

model_en.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model_en.classifier[1].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, NUM_CLASSES)
)

model_en = model_en.to(DEVICE)

Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-bac287d4.pth


100%|██████████| 30.1M/30.1M [00:00<00:00, 197MB/s]


In [ ]:
LR = 0.00001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_en.classifier.parameters(), lr=LR)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
EPOCHS = 25
BATCH_SIZE = 64
PATIENCE = 5
best_val_acc = 0
counter = 0
best_val_f1 = 0
MODEL_PATH = "/content/drive/MyDrive/models/efficientnet_b1_1.pth"

os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

for epoch in range(EPOCHS):

    model_en.train()

    running_loss = 0.0
    all_train_preds = []
    all_train_labels = []

    for i, (inputs, labels) in enumerate(train_loader, 1):

        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model_en(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, 1)

        preds_np = preds.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        all_train_preds.extend(preds_np)
        all_train_labels.extend(labels_np)

        running_loss += loss.item() * inputs.size(0)

        batch_acc = (preds_np == labels_np).mean()
        batch_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)

        sys.stdout.write(
            f"\rEpoch {epoch+1}/{EPOCHS} | "
            f"Batch {i}/{len(train_loader)} | "
            f"Loss: {loss.item():.4f} | "
            f"Acc: {batch_acc:.4f} | "
            f"Prec: {batch_precision:.4f} | "
            f"Rec: {batch_recall:.4f} | "
            f"F1: {batch_f1:.4f}"
        )
        sys.stdout.flush()

    print()

    train_loss = running_loss / len(train_dataset)
    train_acc = np.mean(np.array(all_train_preds) == np.array(all_train_labels))
    train_precision = precision_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_recall = recall_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro', zero_division=0)

    model_en.eval()

    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model_en(inputs)
            preds = torch.argmax(outputs, 1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = np.mean(np.array(val_preds) == np.array(val_labels))
    val_precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    val_f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)

    print(f"\nEpoch {epoch+1} Summary")
    print(f"Train | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_precision:.4f} | Rec: {train_recall:.4f} | F1: {train_f1:.4f}")
    print(f"Val   | Acc: {val_acc:.4f} | Prec: {val_precision:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model_en.state_dict(), MODEL_PATH)
        counter = 0
        print("Model improved. Saved.")
    else:
        counter += 1
        print(f"No improvement. Patience: {counter}/{PATIENCE}")

        if counter >= PATIENCE:
            print("Early stopping triggered")

Epoch 1/25 | Batch 282/282 | Loss: 1.6259 | Acc: 0.5000 | Prec: 0.3000 | Rec: 0.3139 | F1: 0.3000

Epoch 1 Summary
Train | Loss: 1.7167 | Acc: 0.3386 | Prec: 0.3444 | Rec: 0.3386 | F1: 0.3150
Val   | Acc: 0.5195 | Prec: 0.4757 | Rec: 0.5232 | F1: 0.4744
✔ Model improved. Saved.
Epoch 2/25 | Batch 282/282 | Loss: 1.3428 | Acc: 0.6875 | Prec: 0.5643 | Rec: 0.5767 | F1: 0.5633

Epoch 2 Summary
Train | Loss: 1.5506 | Acc: 0.4951 | Prec: 0.5162 | Rec: 0.4951 | F1: 0.4833
Val   | Acc: 0.5682 | Prec: 0.5144 | Rec: 0.5706 | F1: 0.5283
✔ Model improved. Saved.
Epoch 3/25 | Batch 282/282 | Loss: 1.4743 | Acc: 0.3750 | Prec: 0.2472 | Rec: 0.3333 | F1: 0.2836

Epoch 3 Summary
Train | Loss: 1.3983 | Acc: 0.5439 | Prec: 0.5504 | Rec: 0.5439 | F1: 0.5374
Val   | Acc: 0.5763 | Prec: 0.5225 | Rec: 0.5766 | F1: 0.5384
✔ Model improved. Saved.
Epoch 4/25 | Batch 282/282 | Loss: 0.9757 | Acc: 0.7500 | Prec: 0.6667 | Rec: 0.6889 | F1: 0.6593

Epoch 4 Summary
Train | Loss: 1.2895 | Acc: 0.5581 | Prec: 0.559

In [ ]:
print(model_en.features)

Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
  )
  (1): Sequential(
    (0): MBConv(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (activation): SiLU(inplace=True)
          (scale_activation): Sigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), 

Fine tuning by unfreezing last 2 layers of the base model

In [ ]:
for param in model_en.features.parameters():
    param.requires_grad = False
for param in model_en.features[-2:].parameters():
    param.requires_grad = True
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {"params": model_en.classifier.parameters(), "lr": 2e-5},
    {"params": model_en.features[-2:].parameters(), "lr": 2e-6}
], weight_decay=1e-4)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
EPOCHS = 15
BATCH_SIZE = 64
PATIENCE = 5
best_val_acc = 0
counter = 0
best_val_f1 = 0
MODEL_PATH = "/content/drive/MyDrive/models/efficientnet_b1_2.pth"

os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

for epoch in range(EPOCHS):

    model_en.train()

    running_loss = 0.0
    all_train_preds = []
    all_train_labels = []

    for i, (inputs, labels) in enumerate(train_loader, 1):

        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model_en(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, 1)

        preds_np = preds.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        all_train_preds.extend(preds_np)
        all_train_labels.extend(labels_np)

        running_loss += loss.item() * inputs.size(0)

        batch_acc = (preds_np == labels_np).mean()
        batch_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)

        sys.stdout.write(
            f"\rEpoch {epoch+1}/{EPOCHS} | "
            f"Batch {i}/{len(train_loader)} | "
            f"Loss: {loss.item():.4f} | "
            f"Acc: {batch_acc:.4f} | "
            f"Prec: {batch_precision:.4f} | "
            f"Rec: {batch_recall:.4f} | "
            f"F1: {batch_f1:.4f}"
        )
        sys.stdout.flush()

    print()

    train_loss = running_loss / len(train_dataset)
    train_acc = np.mean(np.array(all_train_preds) == np.array(all_train_labels))
    train_precision = precision_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_recall = recall_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro', zero_division=0)

    model_en.eval()

    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model_en(inputs)
            preds = torch.argmax(outputs, 1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = np.mean(np.array(val_preds) == np.array(val_labels))
    val_precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    val_f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)

    print(f"\nEpoch {epoch+1} Summary")
    print(f"Train | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_precision:.4f} | Rec: {train_recall:.4f} | F1: {train_f1:.4f}")
    print(f"Val   | Acc: {val_acc:.4f} | Prec: {val_precision:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model_en.state_dict(), MODEL_PATH)
        counter = 0
        print("Model improved. Saved.")
    else:
        counter += 1
        print(f"No improvement. Patience: {counter}/{PATIENCE}")

        if counter >= PATIENCE:
            print("Early stopping triggered")

Epoch 1/15 | Batch 282/282 | Loss: 0.8884 | Acc: 0.6250 | Prec: 0.6111 | Rec: 0.6667 | F1: 0.6063

Epoch 1 Summary
Train | Loss: 0.9534 | Acc: 0.6464 | Prec: 0.6459 | Rec: 0.6464 | F1: 0.6457
Val   | Acc: 0.6482 | Prec: 0.5892 | Rec: 0.6542 | F1: 0.6078
✔ Model improved. Saved.
Epoch 2/15 | Batch 282/282 | Loss: 0.8239 | Acc: 0.6875 | Prec: 0.7639 | Rec: 0.7639 | F1: 0.6881

Epoch 2 Summary
Train | Loss: 0.9416 | Acc: 0.6477 | Prec: 0.6474 | Rec: 0.6477 | F1: 0.6470
Val   | Acc: 0.6531 | Prec: 0.5959 | Rec: 0.6508 | F1: 0.6149
✔ Model improved. Saved.
Epoch 3/15 | Batch 282/282 | Loss: 1.2014 | Acc: 0.6250 | Prec: 0.4667 | Rec: 0.4944 | F1: 0.4778

Epoch 3 Summary
Train | Loss: 0.9242 | Acc: 0.6571 | Prec: 0.6564 | Rec: 0.6571 | F1: 0.6563
Val   | Acc: 0.6513 | Prec: 0.5930 | Rec: 0.6598 | F1: 0.6129
No improvement. Patience: 1/5
Epoch 4/15 | Batch 282/282 | Loss: 1.2737 | Acc: 0.4375 | Prec: 0.5694 | Rec: 0.4667 | F1: 0.4563

Epoch 4 Summary
Train | Loss: 0.9128 | Acc: 0.6583 | Prec: 